In [ ]:
import numpy as np
import random
import crocoddyl
crocoddyl.switchToNumpyArray()
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
;
np.set_printoptions(linewidth=400, suppress=True)


In [ ]:
starting_configurations = []
optimal_trajectories = []
feasible_trajectories = 0
for _ in range(1000):
    initial_config = np.matrix([random.uniform(-2.1, 2.),
                                random.uniform(-2.1, 2.), 
                                random.uniform(0, 1)]).T

    model = crocoddyl.ActionModelUnicycle()
    #model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
    problem = crocoddyl.ShootingProblem(initial_config, [ model ] * 10, model)
    ddp = crocoddyl.SolverDDP(problem)
    ddp.solve()
    if ddp.isFeasible:
        state = np.squeeze(np.asarray(ddp.xs[1:]))
        control = np.squeeze(np.asarray(ddp.us))
        feasible_trajectories += 1

        optimal_trajectory = np.hstack((state, control))
        starting_configurations.append(np.squeeze(np.asarray(initial_config)))
        optimal_trajectories.append(optimal_trajectory.flatten())
        
starting_configurations = np.asarray(starting_configurations)
optimal_trajectories = np.asarray(optimal_trajectories)   

print(starting_configurations.shape)
print(optimal_trajectories.shape)

In [ ]:


x_train = starting_configurations[0 : 90, :]
y_train = optimal_trajectories[0 : 90, :]
x_test = starting_configurations[90 : 100, :]
y_test = optimal_trajectories[90 : 100, :]
net = Sequential()
net.add(Dense(256, input_dim=(starting_configurations.shape[1])))
net.add(Activation('relu'))
for _ in range(5):
    net.add(Dense(256,
                    activation = "tanh",
                    kernel_initializer='random_uniform',
                    kernel_regularizer=regularizers.l2(0.01),
                    activity_regularizer=regularizers.l1(0.01)))            
    net.add(Dropout(0.25))

net.add(Dense(optimal_trajectories.shape[1], 
                activation = 'linear'))        



sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
net.compile(loss='mean_squared_error',
      optimizer=sgd,
      metrics=['mean_squared_error', "mean_absolute_error"])

print('Train...')

net.fit(x_train, 
          y_train,
          epochs = 10, 
         verbose = 0)

score = net.evaluate(x_test, y_test,  use_multiprocessing=True)
;

In [ ]:
test_state = np.matrix([random.uniform(-2.1, 2.),
                            random.uniform(-2.1, 2.), 
                            random.uniform(0, 1)]).T

model = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem = crocoddyl.ShootingProblem(test_state, [ model ] * 10, model)
ddp = crocoddyl.SolverDDP(problem)
ddp.solve()
test_start = []
crco_trajectory = []
if ddp.isFeasible:
    state = np.squeeze(np.asarray(ddp.xs[1:]))
    control = np.squeeze(np.asarray(ddp.us))
    feasible_trajectories += 1

    optimal_trajectory = np.hstack((state, control))
    test_start.append(np.squeeze(np.asarray(initial_config)))
    crco_trajectory.append(optimal_trajectory.flatten())
        
test_start = np.asarray(test_start)
crco_trajectory = np.asarray(crco_trajectory)   

result = net.predict(test_start)
result = result.reshape(10, 5)
_x = np.array(result[:,0:3])
us = np.array(result[:,3:5])
xs = np.concatenate((np.asarray(test_state).reshape(1,3), _x), axis = 0)

xs_ = xs.tolist()
us_ = us.tolist()
print(len(xs_))
newX = [(i, ) for i in xs]
newU = [(i, ) for i in us]

model_2 = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem_2 = crocoddyl.ShootingProblem(test_state, [ model_2 ] * 10, model)
ddp2 = crocoddyl.SolverDDP(problem_2)
ddp2.solve(init_xs=newX, init_us = newU)
np.set_printoptions(linewidth=400, suppress=True)



In [ ]:
ddp.solve

In [ ]:
type(xs_)

In [ ]:
newl = [(i, ) for i in l]

In [ ]:
test_state

In [ ]:
xs.append(result[i, 0:3 ] for i in range(result.shape[0]))

In [ ]:
us = [result[i, 3:5] for i in range(result.shape[0])]x = np.arange(16.0).reshape(4, 4)

In [ ]:
xs = np.asarray(xs)

In [ ]:
xs

In [ ]:
us = np.asarray(us)
us.shape

In [ ]:
model_2 = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem_2 = crocoddyl.ShootingProblem(test_state, [ model_2 ] * 10, model)
ddp = crocoddyl.SolverDDP(problem_2)
ddp.solve(xs, us)

In [ ]:
x = np.arange(16.0).reshape(4, 4)
np.array_split(x, 3)

In [ ]:
y = np.array(x[:,0:3]) ###############################################

In [ ]:
y

In [ ]:
z = np.vstack([y, test_start])#######################################

In [ ]:
z

In [ ]:
r = np.concatenate((test_start, y), axis = 0) #######################################

In [ ]:
r

In [ ]:
np.matrix(r)

In [100]:
import crocoddyl, numpy as np, random
#crocoddyl.switchToNumpyMatrix()
import eigenpy
eigenpy.switchToNumpyArray()
random.seed(123)
test_state = np.matrix([random.uniform(-2.1, 2.),
                            random.uniform(-2.1, 2.), 
                            random.uniform(0, 1)]).T

model = crocoddyl.ActionModelUnicycle()
model_2 = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem = crocoddyl.ShootingProblem(test_state, [ model ] * 10, model)

problem_2 = crocoddyl.ShootingProblem(test_state, [ model_2 ] * 10, model_2)

ddp = crocoddyl.SolverDDP(problem)

ddp.solve()
print(test_state)



ddp_2 = crocoddyl.SolverDDP(problem_2)
ddp_2.solve(ddp.xs, ddp.us)



[[-1.88530924]
 [-1.74253462]
 [ 0.40724176]]


TypeError: No registered converter was able to produce a C++ rvalue of type Eigen::Matrix<double, -1, 1, 0, -1, 1> from this Python object of type numpy.ndarray

In [96]:
test_state = np.matrix([random.uniform(-2.1, 2.),
                            random.uniform(-2.1, 2.), 
                            random.uniform(0, 1)]).T

model = crocoddyl.ActionModelUnicycle()
#model.costWeights = np.matrix([self.__state_weight, self.__control_weight]).T
problem = crocoddyl.ShootingProblem(test_state, [ model ] * 10, model)
ddp_2 = crocoddyl.SolverDDP(problem)
print()
ddp_2.solve(ddp.xs, ddp.us)
print(test_state)

TypeError: No registered converter was able to produce a C++ rvalue of type Eigen::Matrix<double, -1, 1, 0, -1, 1> from this Python object of type numpy.ndarray

In [59]:
r = np.matrix(ddp.xs)
p = np.matrix(ddp.us)

In [60]:
r.tolist()

[[1.4114707376946556, 0.08444597248933272, 0.12534546335923846],
 [0.55630160815412, -0.02331052851848181, 0.011938178238208197],
 [0.2140842460509912, -0.02739617448087915, -0.007467613213008619],
 [0.08154067733313738, -0.026406371976893585, -0.006864080989344261],
 [0.030839770401071713, -0.02605835137972752, -0.003920502705961962],
 [0.011614085772908775, -0.02598297664493997, -0.0019031051293980678],
 [0.0043660131050017, -0.025969182784014513, -0.0008478511112583293],
 [0.001642955130836623, -0.02596687403573188, -0.0003576210214340153],
 [0.0006230545252728394, -0.025966509297820007, -0.00014556315995623524],
 [0.00024650709576600686, -0.025966454486385907, -5.951345820489934e-05],
 [0.0001224808763789571, -0.025966447105156675, -2.9756729102449755e-05]]

In [64]:
model_2 = crocoddyl.ActionModelUnicycle()
model_2.costWeights = np.matrix([1.5, 2]).T
problem_2 = crocoddyl.ShootingProblem(test_state, [ model_2 ] * 10, model_2)
ddp_ = crocoddyl.SolverDDP(problem_2)
ddp_.solve(init_xs = r.tolist(), init_us = p.tolist())

ArgumentError: Python argument types in
    SolverDDP.solve(SolverDDP)
did not match C++ signature:
    solve(crocoddyl::SolverDDP {lvalue}  self)
    solve(crocoddyl::SolverDDP {lvalue}  self, std::vector<Eigen::Matrix<double, -1, 1, 0, -1, 1>, std::allocator<Eigen::Matrix<double, -1, 1, 0, -1, 1> > >  init_xs=[])
    solve(crocoddyl::SolverDDP {lvalue}  self, std::vector<Eigen::Matrix<double, -1, 1, 0, -1, 1>, std::allocator<Eigen::Matrix<double, -1, 1, 0, -1, 1> > >  init_xs=[], std::vector<Eigen::Matrix<double, -1, 1, 0, -1, 1>, std::allocator<Eigen::Matrix<double, -1, 1, 0, -1, 1> > >  init_us=[])
    solve(crocoddyl::SolverDDP {lvalue}  self, std::vector<Eigen::Matrix<double, -1, 1, 0, -1, 1>, std::allocator<Eigen::Matrix<double, -1, 1, 0, -1, 1> > >  init_xs=[], std::vector<Eigen::Matrix<double, -1, 1, 0, -1, 1>, std::allocator<Eigen::Matrix<double, -1, 1, 0, -1, 1> > >  init_us=[], unsigned int  maxiter=100)
    solve(crocoddyl::SolverDDP {lvalue}  self, std::vector<Eigen::Matr

In [80]:
import numpy as np
import crocoddyl
import example_robot_data
import pinocchio as pin

pin.switchToNumpyMatrix()

def m2a(m): return np.array(m.flat)
def a2m(a): return np.matrix(a).T

# Load robot
robot = example_robot_data.loadTalos()
rmodel = robot.model

# Create data structures
rdata = rmodel.createData()
state = crocoddyl.StateMultibody(robot.model)
actuation = crocoddyl.ActuationModelFloatingBase(state)

# Set integration time
DT = 5e-2
T = 20

# Initialize reference state, target and reference CoM
rightFoot = 'right_sole_link'
leftFoot = 'left_sole_link'
rightFootId = rmodel.getFrameId(rightFoot)
leftFootId =  rmodel.getFrameId(leftFoot)
q0 = rmodel.referenceConfigurations["half_sitting"]
rmodel.defaultState = np.concatenate([q0, np.zeros((rmodel.nv, 1))])
comRef = pin.centerOfMass(rmodel,rdata, q0).copy()
target = np.array([0.4, 0., 1.2])

# Add contact to the model
contactModel = crocoddyl.ContactModelMultiple(state,actuation.nu)
framePlacementLeft = crocoddyl.FramePlacement(leftFootId, pin.SE3.Identity())
supportContactModelLeft = crocoddyl.ContactModel6D(state, framePlacementLeft, actuation.nu, np.matrix([0, 4]).T)
contactModel.addContact(leftFoot + "_contact", supportContactModelLeft)
framePlacementRight = crocoddyl.FramePlacement(rightFootId, pin.SE3.Identity()) 
supportContactModelRight = crocoddyl.ContactModel6D(state, framePlacementRight, actuation.nu, np.matrix([0, 4]).T)
contactModel.addContact(rightFoot + "_contact", supportContactModelRight)
contactData = contactModel.createData(rdata)

# Cost for state, control regularization
uRegCost = crocoddyl.CostModelControl(state,actuation.nu)
#stateWeights = np.array([0] * 3 + [.0] * 3 + [0.5] * 12 + [0.3]*2 + [0.01]* 18 + [1.]*state.nv)
stateWeights = np.array([0.] * 6 + [0.01] * (state.nv-6) + [10.] * state.nv)
xRegCost = crocoddyl.CostModelState(state,crocoddyl.ActivationModelWeightedQuad(np.matrix(stateWeights**2).T),rmodel.defaultState, actuation.nu)
stateWeightsTerm = np.array([0.0] * state.nv + [10.] * state.nv)
xRegTermCost = crocoddyl.CostModelState(state,crocoddyl.ActivationModelWeightedQuad(np.matrix(stateWeightsTerm**2).T),rmodel.defaultState, actuation.nu)

# Cost for target reaching
#Mref = crocoddyl.FrameTranslation(rmodel.getFrameId("gripper_left_joint"), np.matrix(target).T)
Mref = crocoddyl.FrameTranslation(rmodel.getFrameId("wrist_right_tool_joint"), np.matrix(target).T)
goalTrackingCost = crocoddyl.CostModelFrameTranslation(state, Mref)

#Cost for CoM reference
comTrack = crocoddyl.CostModelCoMPosition(state, comRef, actuation.nu)

# Create cost model per each action model
runningCostModel = crocoddyl.CostModelSum(state, actuation.nu)
terminalCostModel = crocoddyl.CostModelSum(state, actuation.nu)

# Then let's added the running and terminal cost functions
runningCostModel.addCost("gripperPose", goalTrackingCost, .1)
runningCostModel.addCost("stateReg", xRegCost, 1e-1)
runningCostModel.addCost("ctrlReg", uRegCost, 5e-5)
#runningCostModel.addCost("comTrack", comTrack, 0)
terminalCostModel.addCost("gripperPose", goalTrackingCost, 1)
#terminalCostModel.addCost("comTrack", comTrack, 0)
terminalCostModel.addCost("stateReg", xRegCost, 1e-6)

# Create the action model
dmodelRunning = crocoddyl.DifferentialActionModelContactFwdDynamics(state, actuation, contactModel, runningCostModel)
dmodelTerminal = crocoddyl.DifferentialActionModelContactFwdDynamics(state, actuation, contactModel, terminalCostModel)
runningModel = crocoddyl.IntegratedActionModelEuler(dmodelRunning, DT)
terminalModel = crocoddyl.IntegratedActionModelEuler(dmodelTerminal,DT)

# Problem definition
x0 = np.concatenate([q0, np.zeros([state.nv,1])])
problem = crocoddyl.ShootingProblem(x0,[runningModel]*T,terminalModel)

# Creating the DDP solver for this OC problem, defining a logger
ddp = crocoddyl.SolverDDP(problem)
ddp.setCallbacks([crocoddyl.CallbackVerbose()])

# pbm=problem
# terminalModel2 = crocoddyl.IntegratedActionModelEuler(dmodelTerminal,DT)
# pbm2 = crocoddyl.ShootingProblem(x0,[runningModel]*T,terminalModel2)
# ddp2 = crocoddyl.SolverFDDP(pbm2)

# stophere
# Solving it with the DDP algorithm
#ddp.solve(ddp.xs,ddp.us,100)



[matrix([[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [

AttributeError: 'list' object has no attribute 'shape'